In [ ]:
# imports
# suppress holoviews warning. Using warnings module didn't work
import logging
logging.getLogger("param.Dimension").setLevel(logging.CRITICAL)
logging.getLogger("param.ParameterizedMetaclass").setLevel(logging.CRITICAL)
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))


import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE

import paths
import functions_bondjango as bd
import functions_misc as fm
import functions_plotting as fp
import pandas as pd
import numpy as np
import sklearn.mixture as mix
import sklearn.decomposition as decomp
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn import svm, datasets
from sklearn import preprocessing

import random
import functions_data_handling as fd

In [ ]:
# Load the data
# define the search string
search_string = 'result:succ, lighting:normal, rig:miniscope, analysis_type:preprocessing, imaging:doric'
# query the database for data to plot
data_all = bd.query_database('analyzed_data', search_string)
# data_path = data_all[0]['analysis_path']

# load the data
# allocate memory
ca_data = []

# for all the entries
for entry in data_all:
    ca_data.append(pd.read_hdf(entry['analysis_path'], 'matched_calcium'))


In [ ]:
# define the number of bins
bin_number = 5;

images = []
scores = []

prediction = []

# # get the mice for this dataset
# mice = data.keys()

# # # for all the mice
# for mouse in mice:
# # for mouse in ['dg_200701_a']:
#     # get the dates for this mouse
#     dates = data[mouse].keys()
# #     # for all the dates
#     for day in dates:
# #     for day in ['2020_09_02']:
#         print(str(mouse)+str(day))
#         # get the table
#         sub_data = data[mouse][day]

# allocate a confusion matrix to average
confusion_overall = np.zeros((bin_number, bin_number))

# for all the files
for idx, sub_data in enumerate(ca_data):
    
    # get the available columns
    labels = list(sub_data.columns)
    cells = [el for el in labels if 'cell' in el]
    not_cells = [el for el in labels if 'cell' not in el]
    # get the cell data
    calcium_data = np.array(sub_data[cells].copy())
#     # get rid of the super small values
    calcium_data[np.isnan(calcium_data)] = 0
    calcium_data[calcium_data<0] = 0
    calcium_data[calcium_data==0] = 1
    calcium_data = np.log(calcium_data)
    print(calcium_data.shape)
#         selection_vector = np.sum(np.isnan(calcium_data), axis=1)==0
#         selection_vector = np.sum(calcium_data>np.std(calcium_data, axis=None)*3, axis=1)==0
#         calcium_data = calcium_data[selection_vector, :]

    if calcium_data.shape[0] == 0 or calcium_data.shape[1] == 0:
        prediction.append([])
        continue
    # scale (convert to float to avoid warning, potentially from using too small a dtype)
#         calcium_data = preprocessing.scale(calcium_data.astype(float), axis=0)
    calcium_data = preprocessing.StandardScaler().fit_transform(calcium_data)

#     print(calcium_data.shape)
    # get the distance to cricket
    distance = sub_data.cricket_0_mouse_distance
#         print(sub_data.keys())
#         distance = sub_data.cricket_0_delta_heading
    # determine the speed bin edges
#         bins = np.linspace(np.min(distance),np.max(distance),num=bin_number)
    bins = np.percentile(distance,np.linspace(0, 100, bin_number+1))   
    bins[-1] = np.ceil(bins[-1])
    
#     bins = [0, 300, 600, 800, 1000, 1200]
    # bin the distance to cricket
    label_vector = np.digitize(distance,bins)-1
    
    print(np.unique(label_vector))
#         label_vector = label_vector[selection_vector]
#     print([np.min(calcium_data.flatten()), np.max(calcium_data.flatten())])
#         frequencies, edges = np.histogram(calcium_data.flatten(), 20)
#         hv.Histogram((edges, frequencies))  
#         print(label_vector.shape)
    # shufle the label vector
    random.shuffle(label_vector)

    # train the classifier
#         linear_pred = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo', random_state=0).fit(calcium_data,label_vector).predict(calcium_data)
    linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr', random_state=0, probability=False)
#         linear.fit(calcium_data, label_vector)
#         linear_pred_save = linear.predict_proba(calcium_data)
#         linear_pred = linear.predict(calcium_data)

    linear_pred_save = cross_val_predict(linear, calcium_data, label_vector, cv=5, verbose=3, 
                                    n_jobs=None, method='decision_function')
#     print(linear_pred_save.shape)
    
#     if len(linear_pred.shape)==1:
#         continue

    linear_pred = np.argmax(linear_pred_save, axis=1)
    confusion_linear = confusion_matrix(label_vector, linear_pred)
    
    if confusion_linear.shape[0] != bin_number:
        prediction.append([])
        continue
    # calculate the accumulated one
    confusion_overall += confusion_linear/len(ca_data)

    # plot
    delta_image = hv.Image(confusion_linear, bounds=[0, 0, confusion_linear.shape[1], confusion_linear.shape[0]])
#         delta_image.opts(width=600, invert_axes=False, invert_yaxis=False, 
#                        invert_xaxis=False, cmap='Viridis', xticks=variable_names, yticks=mouse_data_names, 
#               xrotation=45, tools=['hover'], colorbar=True, shared_axes=False)
    images.append(delta_image)
    scores.append(np.sum(np.diag(confusion_linear))/np.sum(confusion_linear.flatten()))
    prediction.append(linear_pred_save)

In [ ]:
# Plot scores
frequencies, edges = np.histogram(scores, 20)
hv.Histogram((edges, frequencies))

In [ ]:
# Plot the overall confusion matrix
delta_image = hv.Image(confusion_overall, bounds=[0, 0, confusion_linear.shape[1], confusion_linear.shape[0]])
delta_image

In [ ]:
# Plot the classification over time

# Plot classification over time with other variables
print(len(ca_data))
# allocate a dict for the holomap
holo_dict = {}
# select the behavioral variables to show
target_behavior = ['cricket_0_mouse_distance', 'cricket_0_delta_heading', 'prediction']
# define the plot offset
offset = 0
# # define the target mouse, date and trial
# mouse = 'DG_200701_a'
# day = '2020_08_06'
# trial = 1

# define the target trial
target_trial = 503

# # get the table
# if isinstance(data, dict):
#     sub_data = data[mouse][day]
# else:
#     sub_data = data
sub_data = ca_data[target_trial]

# get the available columns
labels = list(sub_data.columns)
cells = [el for el in labels if 'cell' in el]
# calculate the average neuronal response
# get a vector with the trial id

sub_data = sub_data.loc[:, target_behavior + cells]
# sub_data = sub_data.loc[5:, :]

# separate the cells and non-cells
behavior_data = sub_data[target_behavior]
temp_prediction = prediction[target_trial]
# behavior_data['prediction'] = temp_prediction
temp_prediction = np.sort(temp_prediction, axis=1)
behavior_data['prediction'] = temp_prediction[:, -1]/np.sum(temp_prediction[:, :-1], axis=1)
# behavior_data['prediction'] = np.sum(temp_prediction, axis=1)

sub_data = sub_data[cells]
# get the column labels
labels = sub_data.columns
# get the number of rows
x_axis = sub_data.shape[0]
# get the array
sub_array = sub_data.to_numpy().T
sub_array = fm.normalize_matrix(sub_array, axis=1)

# plot the behavioral variables
traces = []
# get the x axis values
x = np.array(np.arange(x_axis))
# get the labels of the behavioral variables
behavior_labels = behavior_data.columns

# # replace the delta head name for heading
# behavior_labels = ['heading' if el=='cricket_0_delta_head' else el for el in behavior_labels]
# behavior_labels = [el.replace('cricket_0_', 'cricket_') if 'cricket_0_' in el else el for el in behavior_labels]

# define the ticks
y_tick = [(idx-0.5, el.replace('_',' ')) for idx, el in enumerate(behavior_labels)]
# for all the behavioral variables
for idx, variable in enumerate(target_behavior):
    
#         # if it's the heading
#     if variable == 'cricket_0_delta_head':
#         # draw a line through the middle of the heading parameter
#         trace2 = hv.Curve((x, 0.5+offset*(idx-1)), 'Time',variable)
#         trace2.opts(width=fp.pix(15), height=fp.pix(6*0.2), xticks=0, xlabel='', yticks=0, ylabel='', 
#                toolbar=None, color='red', line_dash='dotted', line_width=7)
        
#         traces.append(trace2)
    
    y = behavior_data[variable].to_numpy()
    if variable != 'cricket_0_delta_heading':
        y = np.log(y)
        y[np.isnan(y)] = 0
        y[np.isinf(y)] = 0
#     else:
#         np.set_printoptions(precision=1, suppress=True)
#         print(y)

    y[np.isnan(y)] = 0
    y = fm.normalize_matrix(y, axis=0)+offset*(idx-1)
    
    trace = hv.Curve((x, y), 'Time',variable)
#     trace = hv.Curve((x, behavior_data[variable].to_numpy()+offset*(idx-1)), 'Time',variable)
#     trace.opts(width=fp.pix(15), height=fp.pix(8*0.3), xticks=0, xlabel='', yticks=y_tick, ylabel='', 
#                toolbar=None, color='black', line_width=7)
    trace.opts(width=800)
#     if variable == 'mouse_speed':
#         print(fm.normalize_matrix(np.log(behavior_data[variable].to_numpy()), axis=0))

    # append
    traces.append(trace)
        
# plot the cells
raw_image = hv.Image(sub_array, ['Time (s)','Cells'], bounds=[0,-0.5,x_axis,0.5], fontstyle='regular')
raw_image.opts(width=800)
# raw_image.opts(width=fp.pix(15), height=fp.pix(8*0.7), colorbar=True, colorbar_opts={'major_label_text_align': 'left'},
#                cmap='Viridis', yticks=[(-0.5, 1), (0.5, sub_array.shape[0])], toolbar=None,fontsize=fp.font_sizes['small'], 
#                hooks=[fp.margin])

# create the overlay with the behavioral traces
overlay = hv.Overlay(traces)
# overlay.opts(text_font='Arial',fontsize=fp.font_sizes['small'])

# create the final layout
layout = hv.Layout(raw_image+overlay).cols(1)
layout.opts(fontsize=fp.font_sizes['small'])

# # assemble the save path
# save_path = os.path.join(paths.figures_path,'_'.join((save_name,mouse,day,str(trial),'ethogram.png')))
# hv.save(layout,save_path)

layout
# print(sub_array)